In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import codecs,os,time

In [ ]:
import gensim
from gensim.models import Phrases, Word2Vec

In [ ]:
from sklearn import preprocessing 

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import cross_val_score,cross_val_predict
from sklearn.pipeline import Pipeline

from sklearn.model_selection import ShuffleSplit,train_test_split 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import shuffle

from sklearn.svm import LinearSVC,NuSVC
from sklearn.linear_model import SGDClassifier ,PassiveAggressiveClassifier,Perceptron,LogisticRegression
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import Normalizer,FunctionTransformer,MinMaxScaler

In [ ]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import SVG

In [ ]:
from keras.callbacks import EarlyStopping,Callback
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

from keras.layers import Dense,Activation,Dropout, LSTM, Bidirectional,TimeDistributed,GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence,text as Text

from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.utils import vis_utils #import plot_model, model_to_dot

In [ ]:
def plot_model(model):
    return SVG(vis_utils.model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
def scores(y_test,y_pred):
    print 
    print("Accuracy: %.2f%%" % (accuracy_score(y_test,y_pred)*100))
    print("Precision: %.2f%%" % (precision_score(y_test,y_pred)*100))
    print("Recall: %.2f%%" % (recall_score(y_test,y_pred)*100))
    print("F1-score: %.2f%%" % (f1_score(y_test,y_pred)*100))

In [ ]:
earlystopping_patience = [EarlyStopping(monitor='val_loss',patience=2)]
earlystopping_littlepatience = [EarlyStopping(monitor='val_loss',patience=1)]
earlystopping = earlystopping_nopatience = [EarlyStopping(monitor='val_loss',patience=0)]

In [ ]:
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337], [1, 3, 7, 9, 2, 1337, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for i in range(len(new_list)-ngram_range+1):
            for ngram_value in range(2, ngram_range+1):
                ngram = tuple(new_list[i:i+ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

ngram_range = 2
print('Adding {}-gram features'.format(ngram_range))

### Cleaning utf-8 encoding issue

In [ ]:
def cleanTweets(texts):
    cleanedTexts = []
    for text in texts:
        cleanedTexts += [unicode(str(text),errors='ignore').encode("latin-1","ignore").replace('\r','')]
    return cleanedTexts

### Reverse word order 

In [ ]:
def reverseText(text):
    words=text.split()
    # reverse list
    words.reverse()
    # now print
    return " ".join(words)

### Finding Maximum length of a tweet 

In [ ]:
def find_max_length(texts=tweets):
    maxLength = -1
    for text in texts:
        if len(text)>maxLength:
            maxLength = len(text)
    return maxLength

### Plotting test-train loss and accuracy 

In [ ]:
def plotTestTrainCurves(history):
    print "History Summary for accuracy"
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    print "History Summary for loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
def prepare_train_test_split(X=tweets,y=labels):
    X = array(X)
    y = array(y)
    ###################################
    X,y = shuffle(X,y,random_state=10)
    X,y = shuffle(X,y,random_state=0)
    ###################################
    tk = Text.Tokenizer(lower=True)
    tk.fit_on_texts(X)

    x = tk.texts_to_sequences(X)

    ###################################
    max_len = 174
    embedding_vecor_length = 32
    print "max_len ", max_len
    print('Pad sequences (samples x time)')

    x = sequence.pad_sequences(x, maxlen=max_len)

    return train_test_split(x,y, test_size=0.2,stratify=y)

## <font color="green"> Embedding Models </font>

### GloVe

In [ ]:
with open("glove/glove.6B.50d.txt", "rb") as lines:
    gloveW2VWiki = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [ ]:
with open("glove/glove.twitter.27B.50d.txt", "rb") as lines:
    gloveW2VTwitter = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [ ]:
with open("glove/glove.42B.300d.txt", "rb") as lines:
    gloveW2VCommonCrawlSmall = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [ ]:
with open("glove/glove.840B.300d.txt", "rb") as lines:
    gloveW2VCommonCrawlBig = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

### Word2Vec

In [ ]:
def Words(sentences):
    words = []
    for sentence in sentences:
        tokens =  word_tokenize(sentence)
        words +=[tokens]
    return words

In [ ]:
tweetCorpus = cleanTweets(open('profane_tweets.txt').readlines())

In [ ]:
unigram_transformer = Words(tweetCorpus)
bigram_transformer = Phrases(tweetCorpus)

In [ ]:
embedding_folder = "embeddingModels/"

### CBOW

In [ ]:
cbow_word_model = Word2Vec(unigram_transformer, size=100)
CBOW_word_W2V = dict(zip(cbow_word_model.wv.index2word, cbow_word_model.wv.syn0))
cbow_word_model.save(embedding_folder + 'profanity_CBOW.word2vec')

In [ ]:
cbow_phrase_model = Word2Vec(bigram_transformer[tweetCorpus], size=100)
CBOW_phrase_W2V = dict(zip(cbow_phrase_model.wv.index2word, cbow_phrase_model.wv.syn0))
cbow_phrase_model.save(embedding_folder + 'profanity_CBOW.phrase2vec')

### Skipgrams

In [ ]:
skipgram_word_model = Word2Vec(unigram_transformer, sg=1, size=100)
skipgram_word_W2V = dict(zip(skipgram_word_model.wv.index2word, skipgram_word_model.wv.syn0))
skipgram_word_model.save(embedding_folder + 'profanity_Skipgram.word2vec')

In [ ]:
skipgram_phrase_model = Word2Vec(bigram_transformer[tweetCorpus], sg=1, size=100)
skipgram_phrase_W2V = dict(zip(skipgram_phrase_model.wv.index2word, skipgram_phrase_model.wv.syn0))
skipgram_phrase_model.save(embedding_folder + 'profanity_Skipgram.phrase2vec')

In [ ]:
def generateEmbeddingMatrix(word_vectors, embedding, word_index, vocabulary_size=vocab_size):
    NUM_WORDS = vocabulary_size
    embedding_dim = len(embedding[embedding.keys()[0]])
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    for word, i in word_index.items():
        if i>=NUM_WORDS:
            continue
        try:
            embedding_vector = word_vectors[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
    return embedding_matrix

    

In [ ]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [ ]:
embeddings_index = {}
word_index = tokenizer.word_index

for line in open('glove/glove.twitter.27B.50d.txt'):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

glove_twitter_embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_twitter_embedding_matrix[i] = embedding_vector

In [ ]:
embeddings_index = {}
word_index = tokenizer.word_index

for line in open('glove/glove.twitter.27B.200d.txt'):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

glove_twitterBig_embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_twitterBig_embedding_matrix[i] = embedding_vector

#### Google Pretrain Vectors

In [ ]:
EMBEDDING_DIM=300
NUM_WORDS = vocab_size
google_embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        google_embedding_vector = google_pretrained_vectors[word]
        google_embedding_matrix[i] = google_embedding_vector
    except KeyError:
        google_embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(google_pretrained_vectors)

### <font color="teal">Pretrain Embedding Models</font>

In [ ]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_profane_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
def classifier( classifier='sgd',ngram=2,idf=True, X = tweets, y=labels):
    start_time = time.time()
    vectorizer = TfidfVectorizer(input='content', min_df=3,strip_accents='unicode', analyzer ='word', tokenizer=LemmaTokenizer(), ngram_range=(1, ngram), use_idf=idf, sublinear_tf=1, stop_words = "english")
    clf = None
    if classifier =="votingBestHard":
        clf =  Pipeline([('vect', vectorizer), VotingClassifier(estimators=[ ('pa', PassiveAggressiveClassifier()),('et', ExtraTreesClassifier(n_estimators=200)),('rf', RandomForestClassifier())])])
        pass
    elif classifier == "votingBestSoft":
        clf = VotingClassifier(estimators=[])
        pass
    elif classifier=='naive':
        clf =  Pipeline([('vect', vectorizer), ('nb', MultinomialNB())])
    elif '3nn' in classifier:
        clf =  Pipeline([('vect', vectorizer),('3nn',KNeighborsClassifier(n_neighbors=3))])
    elif '5nn' in classifier:
        clf =  Pipeline([('vect', vectorizer),('5nn',KNeighborsClassifier(n_neighbors=5))])
    elif 'logistic' in classifier:
        clf = Pipeline([('vect', vectorizer), ('logistic', LogisticRegression(solver='newton-cg'))])
    elif 'perceptron' in classifier:
        clf = Pipeline([('vect', vectorizer), ('perceptron', Perceptron())])
    elif 'passiveaggressive' in classifier:
        clf = Pipeline([('vect', vectorizer), ('pa', PassiveAggressiveClassifier())])
    elif 'paGlove' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2V)), ('pa', PassiveAggressiveClassifier())])
    elif 'paGensim' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(bigramW2V)), ('pa', PassiveAggressiveClassifier())])
    elif 'paSkipgram' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(skipBigramW2V)), ('pa', PassiveAggressiveClassifier())])
    elif 'bernoulliTfidf' in classifier:
        clf = Pipeline([('vect', vectorizer), ('bnb', BernoulliNB(binarize=0.0))])
    elif 'bernoulliGlove' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2VWiki)), ('bnb', BernoulliNB(binarize=0.0))])
    elif 'bernoulliGensim' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(bigramW2V)), ('bnb', BernoulliNB(binarize=0.0))])
    elif 'svcTfidf' in classifier:
        clf = Pipeline([('vect', vectorizer), ('svc', LinearSVC())])
    elif 'svcGlove' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2VWiki)), ('svc', LinearSVC())])
    elif 'svcGensim' in classifier:
        clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(bigramW2V)), ('svc', LinearSVC())])
    elif 'randomforest' in classifier:
        clf = Pipeline([('vect', vectorizer), ('rf', RandomForestClassifier(n_jobs=-1))])
    elif 'extraTrees' in classifier:
        clf = Pipeline([('vect', vectorizer), ('et', ExtraTreesClassifier(n_estimators=200))])
    elif 'lsa' in classifier:
        clf = Pipeline([('countvect',CountVectorizer(min_df = 1, stop_words = 'english')),('svd',TruncatedSVD(2, algorithm = 'arpack')),('norm',Normalizer(copy=False)),('et', ExtraTreesClassifier(n_estimators=200))])
    elif 'lsaRandom' in classifier:
        clf = Pipeline([('countvect',CountVectorizer(min_df = 1, stop_words = 'english')),('svd',TruncatedSVD(2, algorithm = 'arpack')),('norm',Normalizer(copy=False)),RandomForestClassifier()])
    
    elif 'gloveWiki' in classifier:
        if not idf:
            clf = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(gloveW2VWiki)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
        else: #'tflove' in classifier: #glove with tfidf
            clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2VWiki)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
    
    elif 'gloveTwitter' in classifier:
        if not idf:
            clf = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(gloveW2VTwitter)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
        else: #'tflove' in classifier: #glove with tfidf
            clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2VTwitter)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
    
    elif 'gloveCommonCrawlBig' in classifier:
        if not idf:
            clf = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(gloveW2VCommonCrawlBig)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
        else: #'tflove' in classifier: #glove with tfidf
            clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2VCommonCrawlBig)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
    
    elif 'gloveCommonCrawlSmall' in classifier:
        if not idf:
            clf = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(gloveW2VCommonCrawlSmall)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
        else: #'tflove' in classifier: #glove with tfidf
            clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(gloveW2VCommonCrawlSmall)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
    
    elif 'cbow' in classifier:
        if ngram==1:
            if not idf:
                clf = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(CBOW_Unigram_W2V)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
            else:
                clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(CBOW_Unigram_W2V)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
        else:
            if not idf:
                 clf = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(CBOW_Bigram_W2V)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
            else:#'tfbigram' in classifier:
                 clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(CBOW_Bigram_W2V)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
    elif 'skipgram' in classifier:
        if ngram==1: 
             clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(skipgrams_Unigram_W2V)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
        else: 
             clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(skipgrams_Bigram_W2V)),("extra trees", ExtraTreesClassifier(n_estimators=200))])
    
    elif 'skipRandom' in classifier:
        if ngram==1: 
             clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(skipgrams_Unigram_W2V)),('rf', RandomForestClassifier())])
        else: 
             clf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(skipgrams_Bigram_W2V)),('rf', RandomForestClassifier())])
    
    elif 'adaBoost' in classifier:
        clf = Pipeline([('vect', vectorizer), ('sgd', AdaBoostClassifier())])
    
    elif 'sgd' in classifier:
        clf = Pipeline([('vect', vectorizer), ('sgd', SGDClassifier(class_weight="auto"))])
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,stratify=y)
    clf.fit(X_train,y_train)
    y_predicted = clf.predict(X_test)
    scores(y_test,y_predicted)
    stop_time = time.time()
    print("Running Time is %.2fseconds" % (stop_time - start_time))
    return 
    

In [ ]:
df = pd.read_csv('agressive_tweet_classed.txt',sep='==>',encoding='utf-8')

In [ ]:
df.head()

In [ ]:
tweets = cleanTweets(df['tweet'].tolist())
labels = df['class'].tolist()

In [ ]:
tweetsTiny = tweets[:2000]
tweetsSmall = tweets[:8000]

labelsTiny = labels[:2000]
labelsSmall = labels[:8000]


In [ ]:
tokenizer = Text.Tokenizer()
tokenizer.fit_on_texts(tweets)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
max_profane_length = find_max_length()
max_profane_length

In [ ]:
embedding_vector_length = 32

### <font color="skyblue">Data Visualization </font>

In [ ]:
# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [tweet_w2v[w] for w in tweet_w2v.wv.vocab.keys()[:5000]]

# dimensionality reduction. converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = tweet_w2v.wv.vocab.keys()[:5000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

### <font color="bieg"> Codec conversion to handle tweets </font>

In [ ]:
tweets[78]

In [ ]:
text = 'Female Killer In Las Vegas Shouted \x81\xe6\xe4\xf3\xee\xe4\xf3_Allahu Akbar\x81\xe6\xe4\xf3\xee\xed\x9a As She Ran Over 40 Innocent People Last Night'

In [ ]:
unicode(text,errors='ignore').encode("latin-1","ignore")

In [ ]:
nonUtfIds[1],unicode(texts[nonUtfIds[1]],errors='ignore').encode("latin-1","ignore")

### <font color="red"> Traditional Machine Learning </font>

In [ ]:
classifier('naive')

In [ ]:
classifier('logistic')

In [ ]:
classifier('logistic',ngram=2)

In [ ]:
classifier('randomforest')

In [ ]:
classifier('randomforest',ngram=2)

In [ ]:
classifier('randomforest',X = tweetsSmall, y=labelsSmall)

In [ ]:
classifier('randomforest',X = tweetsTiny, y=labelsTiny)

#### GloVe

In [ ]:
classifier('gloveWiki')

In [ ]:
classifier('gloveWiki',idf=False)

In [ ]:
classifier('gloveTwitter')

In [ ]:
classifier('gloveTwitter',idf=False)

In [ ]:
classifier('gloveCommonCrawlBig')

In [ ]:
classifier('gloveCommonCrawlBig',idf=False)

In [ ]:
classifier('gloveCommonCrawlSmall')

In [ ]:
classifier('gloveCommonCrawlSmall',idf=False)

#### Word2Vec: CBOW

In [ ]:
classifier('cbow')

In [ ]:
classifier('cbow',ngram =2)

In [ ]:
classifier('cbow',idf=False)

In [ ]:
classifier('cbow',ngram =2, idf=False)

#### Word2Vec: Skipgrams 

In [ ]:
classifier('skipgrams')

In [ ]:
classifier('skipgrams',ngram=2)

In [ ]:
classifier('skipgrams',idf=False)

In [ ]:
classifier('skipgrams',ngram=2, idf=False)

### Other Methods 

In [ ]:
classifier('bernoulliTfidf')

In [ ]:
classifier('bernoulliGlove')

In [ ]:
classifier('3nn')

In [ ]:
classifier('5nn')

In [ ]:
classifier('extraTrees')

In [ ]:
classifier('extraTrees',ngram=2)

In [ ]:
classifier('extraTrees',X = tweetsSmall, y=labelsSmall)

In [ ]:
classifier('extraTrees',X = tweetsTiny, y=labelsTiny)

In [ ]:
classifier('svcGlove',idf=False)

In [ ]:
classifier('sgd')

### <font color="magenta"> Model Evaluation: Misclassified & Confusion Matrix</font>
http://www.ritchieng.com/machine-learning-evaluate-classification-model/

In [ ]:
vectorizer = TfidfVectorizer(input='content', min_df=3,strip_accents='unicode', analyzer ='word', tokenizer=LemmaTokenizer(), ngram_range=(1, 2), use_idf=True, sublinear_tf=1, stop_words = "english")
logistic_model = Pipeline([('vect', vectorizer), ('logistic', LogisticRegression(solver='newton-cg'))])
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, test_size=0.2)
logistic_model.fit(X_train,y_train)
y_predicted = logistic_model.predict(X_test)


In [ ]:
scores(y_test,y_predicted)

In [ ]:
countFP,countFN = 0,0
FP, FN = [],[]
for i in range(len(y_predicted)):
    if y_predicted[i] != y_test[i]:
        if y_test[i] == 0 and y_predicted[i] == 1:
            FP += [X_test[i]]
            countFP += 1
        else:
            FN += [X_test[i]]
            countFN += 1
            
countFP, countFN, len(y_predicted)

### <font color='bieg'> Examples that are misclassified as offensive </font>

In [ ]:
FP[:10]

### <font color="orange"> Examples that are misclassified as non-offensive </font>

In [ ]:
FN[20:40]

## <font color="blue"> Deep Neural Networks </font>

### <font color="brown">LSTMs without Dropout</font>

In [ ]:
max_profane_length = 174
embedding_vector_length = 32
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, test_size=0.2)
# X_train = sequence.pad_sequences(X_train, maxlen=max_profane_length)
# X_test = sequence.pad_sequences(X_test, maxlen=max_profane_length)

In [ ]:
model = Sequential()
optimizer = optim(lr=0.001)
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_profane_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

### <font color="green"> LSTMs with Dropout </font>

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_len))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()

#### <font color="darkblue">Batch size: 32</font>

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

#### <font color="olive">Batch size: 64</font>

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plotTestTrainCurves(history)

#### LR = 10^-3 (with earlystopping_patience)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.001), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.1, callbacks = earlystopping_patience, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### with little patience (patience = 1)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.001), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks = earlystopping_littlepatience,shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### with little patience = 1 

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.0005), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks = earlystopping_littlepatience,shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.0002), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks = earlystopping_littlepatience,shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### with patience and lr=2x10^-4

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.0002), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks = earlystopping_patience,shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### LR = 10^-4 (with  earlystopping_patience)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.0001), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks = earlystopping_patience, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### LR = 10^-2

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(lr=0.01), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### <font color="RoyalBlue">Reversed tweets</font>

In [ ]:
reversedTweets = []
for tweet in tweets:
    reversedTweets += [reverseText(tweet)]

#### Batch Size : 32

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split(X=reversedTweets)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, callbacks=earlystopping_nopatience,verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### Batch Size : 64

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split(X=reversedTweets)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, callbacks=earlystopping_nopatience,verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plotTestTrainCurves(history)

#### Batch Size : 128

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split(X=reversedTweets)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, callbacks=earlystopping_nopatience,verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### Small Dataset (Size = 8000)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split(X=tweetsSmall,y=labelsSmall)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split(X=tweetsSmall,y=labelsSmall)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

#### Tiny Dataset (Size = 2000)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split(X=tweetsTiny,y=labelsTiny)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split(X=tweetsTiny,y=labelsTiny)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

### With patience

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, callbacks =earlystopping_patience, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, callbacks =earlystopping_patience, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_len, trainable=True ))
model.add(Dropout(0.2))
model.add(LSTM(200))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1,callbacks=earlystopping, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves()

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_len, trainable=True ))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1,callbacks=earlystopping, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves()

### <font color="royalblue">Two Layered LSTMs </font>

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length, trainable=True ))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, callbacks=earlystopping, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

### <font color="green"> LSTMs with glove - Twitter </font>

In [ ]:
model = Sequential()
    
model.add(Embedding(vocab_size, 50,  weights=[glove_twitter_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, callbacks=earlystopping, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 200,  weights=[glove_twitterBig_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, callbacks=earlystopping, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

## <font color="maroon"> LSTMs with twitter word2vec </font>

### Continous Bag of Words 

In [ ]:
embedding_matrix = generateEmbeddingMatrix(cbow_word_model, CBOW_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(CBOW_word_W2V[CBOW_word_W2V.keys()[0]])

In [ ]:
model = Sequential()
    
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

### Skipgrams 

In [ ]:
embedding_matrix = generateEmbeddingMatrix(skipgram_word_model, skipgram_word_W2V, tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(skipgram_word_W2V[skipgram_word_W2V.keys()[0]])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plotTestTrainCurves(history)

### Phrase2Vec

In [ ]:
embedding_matrix = generateEmbeddingMatrix(skipgram_phrase_model, skipgram_phrase_W2V, tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(skipgram_word_W2V[skipgram_word_W2V.keys()[0]])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plotTestTrainCurves()

### Applying Embeddings


1. https://www.bonaccorso.eu/2016/08/02/reuters-21578-text-classification-with-gensim-and-keras/
2. https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html
3. https://www.kaggle.com/marijakekic/cnn-in-keras-with-pretrained-word2vec-weights

#### More resources:
1. https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
2. https://machinelearningmastery.com/improve-deep-learning-performance/
3. http://www.ritchieng.com/machinelearning-learning-curve/

### <font color="red"> GRUs </font>

### <font color="purple"> LSTMs with CNN </font>

In [ ]:
top_words=5000
embedding_vecor_length = 32

In [ ]:
embedding_matrix = generateEmbeddingMatrix(cbow_word_model, CBOW_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(CBOW_word_W2V[CBOW_word_W2V.keys()[0]])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.summary()

#### <font color="brown">Batch size : 32</font>

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=40, verbose=1, validation_split=0.1, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plotTestTrainCurves()

#### <font color="magenta">Batch size:64</font>

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=40, verbose=1, validation_split=0.1, batch_size=64,callbacks=earlystopping_patience)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### <font color='bieg'>Batch Size 128 </font>

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=40,verbose=1,validation_split=0.1, batch_size=128,callbacks=earlystopping_patience)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves()

### <font color="red">Bidirectional LSTMs</font>

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

### <font color="bieg">Fast Text </font>
https://martinbel.github.io/fast-text.html

In [ ]:
def optim(optimizer='adam',lr=0.001):
    if optimizer == 'adam':
        optim = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    elif optimizer == 'rmsprop':
        optim = RMSprop(lr=lr, rho=0.9, epsilon=1e-08, decay=0.0)
    elif optimizer == 'sgd-momentum':
        optim = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    elif optimizer == 'adagrad':
        optim = Adagrad(lr=lr, epsilon=1e-08, decay=0.0)
    return optim

In [ ]:
model = Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions

model.add(Embedding(vocab_size, embedding_vector_length,  input_length=max_profane_length))
# we add a GlobalAveragePooling1D, which will average the embeddings
# of all words in the document

model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optim(), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
predicted = model.predict_classes(X_test)
scores(y_test,predicted)
plotTestTrainCurves(history)

In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test_split()
model.compile(loss='binary_crossentropy', optimizer=optim(), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### <font color="magenta">Skipgram (Negative Sampling) with FastText</font>

In [ ]:
skipgramNeg_word_model = Word2Vec(unigram_transformer, sg=1, size=100,negative=20)
skipgramNeg_word_W2V = dict(zip(skipgramNeg_word_model.wv.index2word, skipgramNeg_word_model.wv.syn0))

In [ ]:
embedding_matrix = generateEmbeddingMatrix(skipgramNeg_word_model, skipgramNeg_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(skipgramNeg_word_W2V[skipgramNeg_word_W2V.keys()[0]])

In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
# model.add(GlobalAveragePooling1D())
# model.add(Dense(1, activation='sigmoid'))
# print 'Fast Text Summary'
# model.summary()
# model.compile(loss='binary_crossentropy', optimizer=optim(), metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### <font color="purple">FastText with Twitter GloVe</font>

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50,  weights=[glove_twitter_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
print 'Fast Text Summary'
model.summary()
model.compile(loss='binary_crossentropy', optimizer=optim(), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=64, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)